# **HOPOMO**




**HoPoMo** est une modélisation particulière des dynamiques d'une colonie d'abeille au sein de leur ruche ainsi que le management des ressources qui y sont liées. Elle a été pensée par **Thomas Schmickl** et **Karl Crailsheim**, qui ont vu leur article publié en 2007 par l'éditeur *ELSEVIER*.

Ainsi à l'aide d'un modèle équationnel d'environ **60 équations** nous modéliserons à travers celui-ci le développement des individus au sein de la ruche ainsi que la gestion de ses besoins, ses stocks et productions. 

De plus, nous avons grandement amélioré la précision des résultats en modifiant ce modèle de plusieurs manières : entre autre les **facteurs météorologiques**.

L'implémentation de ce modèle se terminera par une série de **graphiques** modélisant les variables clées de celui-ci.

## **IMPLEMENTATION**

L'implémentation suivante a été réalisée avec le langage **PYTHON 3** sous l'éditeur de Notebook **Google Colab**.

### **IMPORTS**

In [2]:
# Importation du module Astral
# Documentation : https://astral.readthedocs.io/en/latest/index.html

!pip install astral

In [3]:
#Les différents imports nécessaires au fonctionnement du programme.

from functools import lru_cache # permet de mettre en cache les calculs récursifs afin de soulager grandement la machine et d'optimiser les calculs
from plotly.subplots import make_subplots # générateur de graphes dynamiques
from plotly import tools # https://github.com/plotly/plotly.py
from itertools import islice # https://docs.python.org/3/library/itertools.html
from astral import *
from astral.sun import *
from datetime import * # https://docs.python.org/3/library/datetime.html
from pprint import pprint #https://docs.python.org/3/library/pprint.html
import gc #https://docs.python.org/3/library/gc.html

import datetime as dt
import numpy as np # https://numpy.org/doc/stable/
import pandas as pd # https://pandas.pydata.org/docs/user_guide/index.html
import random as rd # https://docs.python.org/3.9/library/random.html
import plotly.graph_objs as go 
import requests # https://requests.readthedocs.io/en/master/
import json # https://docs.python.org/3/library/json.html

### **PARAMETRES**



Concernant les paramètres nous utilisons majoritairement comme référencement les pages de l'article officiel de la modélisation HoPoMo.

Vous pouvez l'obtenir via ce lien : https://www.sciencedirect.com/science/article/abs/pii/S0304380007000075

Obtention possible par achat ou utilisation d'une raison institutionnelle, de recherche.

In [4]:
# Comportements de la reine/ponte
x1, x2, x3, x4, x5 = 385, 30, 36, 155, 45 #REF p230 # x4 = jour de l'année où la reine pond le plus, 
                                                    # x5 = courbure "à droite" de la fonction saison plus on l'augmente plus la courbe favorable s'aplanit (permet de modifier la "durée" de l'été)
ELRbase = 1600 #REF p230 - taux de ponte moyen de la reine (dépend de la reine, son âge etc ..)
SUPtreshold = 0.2 #REF p230 # si le taux de cellules vides est inférieur à cette limite alors il y a une diminution du taux de ponte (la reine a de plus en plus de mal à trouver des cellules disponibles)
ELRstochrange = 0 # facteur stochastique du taux de ponte de la reine

# Cellules
CELLShive = 250000 #REF p230 # nb de cellules dans la ruche

# Initialisations
BEESadultINIT = 15000
STOREShoneyINIT = 50000

# Couvain
LIFESPANegg = 3 #REF p230 #durée max (en jours) de l'état oeuf
LIFESPANlarvae = 5 #REF p230 #durée max (en jours) de l'état larve
LIFESPANpupae = 12 #REF p230 #durée max (en jours) de l'état pupe 
MORTALITYeggs = 0.03 #REF p230 # Mortalités (Plus le stade est avancé et moins le taux de mortalité est élevé)
MORTALITYlarvae = 0.01 #REF p230
MORTALITYpupae = 0.001 #REF p230
CANNIBALISMbase = [0.23,0.30,0.58,0.06,0] #REF p230 # Taux fixe de cannibalisme des larves en fonction de leur age (1-5 jours)

# Abeilles adultes
MORTALITYadultbase = 0.01 #REF p230
MORTALITYnursing = 0.005 #REF p230
MORTALITYprocessing = 0.005 #REF p230
MORTALITYforaging = 0.035 #REF p230

# Facteurs liés aux ressources
FACTORpollensavingmax = 0.30 #REF p230
RATIOnectar_to_honey = 0.4 #REF p230 20/50
FACTORothertasks = 0.2 #REF p225
FACTORpollenstorage = 6 #REF p230 - La colonie régule le stockage du pollen autour d'un niveau qui représente une réserve approximative de 6 jours, basé sur la demande actuelle.
FACTORminpollenforagers = 0.01 #REF p230
FACTORforagingmax=0.33 #REF p230

# Besoins
NECTARNEEDactiveforager = 0.03 # 
NECTARNEEDnurse = 0.02 # 
NECTARNEEDadult = 0.005 #
NECTARNEEDlarva = [0.0025,0.0050,0.0084,0.016,0.032] #REF p231 graph b
POLLENNEEDlarva = [0.0150,0.0250,0.0400,0.080,0.156] #REF p231 graph b
POLLENNEEDadult = 0.0037 # inspiré de Rortais et al. (2005) article
POLLENNEEDnurse = 0.045 # inspiré de Rortais et al. (2005) article
NEEDnurses_per_larva = [0.10,0.30,0.70,1.70,3.0] #REF p231 graph d # à déterminer, augmentation exponentielle grandissant avec la masse de la larve
NEEDnurses_per_egg = 0.30 # à établir, moins important que pour les larves car pas besoin d'être nourrit
NEEDnurses_per_pupa = 0.30 # à établir, moins important que pour les larves car pas besoin d'être nourrit

#Butinage
LOADpollenforager = 0.06 # cellfulls REF p230
LOADnectarforager = 0.04 # cellfulls REF p230
FACTORforagingsuccess = 0.8 #REF p230
TURNSnectarforager = 15 #REF p230
TURNSpollenforager = 10 #REF p230
ProcessorsPerCell = 2  #REF p230
stochasticForagingFactor = 0.25

# Poids (en grammes)
w_hivebase = 14000   #REF p231
w_cellsbase = 0.037  #REF p231 #represents the weight of one empty cell (wax only).
w_pollen = 0.23  #REF p231 # poids d'une cellule remplie de pollen
w_nectar = 0.43  #REF p231 # poids d'une cellule remplie de nectar
w_honey = 0.50  #REF p231 #poids d'une cellule remplie de miel
w_egg = 0.0001  #REF p231
w_larva = [0.0002,0.00059,0.00331,0.0644,0.160] #le poids d'une larve à l'état i ∈ {1,2,3,4,5}
w_pupa = 0.16  #REF p231
w_adult = 0.10  #REF p231

max_wind = 45 #km/h 
min_humidity = 60 #pourcentage

nb_days = 364

### **METEOROLOGIE ET TRAITEMENTS**

In [ ]:
# Import des données des CSV
data2020 = pd.read_csv("/content/meteo_gelos_owm_2020.csv",skiprows=[i for i in range(1394,1407)],sep =',') #exclusion 29/02 pour simplifier les processus 

# Données globales
dataMeteo2020 = pd.DataFrame(data2020,columns=['date','main.temp','rain.1h','main.humidity','wind.speed'])

# Verifications du jeu de données
dataMeteo2020.isnull().sum()

# Correction du jeu de données avec un median (la correction n'est faite ici que sur les trous de données des précipitations)
# Si besoin copiez coller les deux lignes suivantes en changeant le nom de la colonne entre [ ] avec celle concernée (trou de données visibles avec l'appel précédent)
median = dataMeteo2020['rain.1h'].median()
dataMeteo2020['rain.1h'].fillna(median,inplace=True)

#Conversion en listes des données météo
list_DataMeteo2020 = [list(row) for row in dataMeteo2020.values]

In [744]:
# Traitement des listes pour le modèle

# Conversion de la date et de l'heure puis transformation de ces deux variables
# en des listes.

# Cette fonction prends en compte le traitement de trois cas (données sources) différents de données. 
# Afin de pouvoir transformer les colonnes date et heure en listes
# normalisées de la manière suivante [année,mois,jour],[heure,minutes,secondes]

# Cette fonction traite ainsi les données d'historique de 2020, de 2021 et celles de prévision
# listeIn est une liste de sous listes et valeurs contenant toutes les données récupérées des csv ou json
# tel que pour un jour on ait en sortie : [[année, mois, jour],[heure, minutes, secondes], temperature, pluie, humidité, vent]

def convert_date(listeIn) :
  for i in range(len(listeIn)):
    if ('T' in listeIn[i][0]):
      listeIn[i][0] = listeIn[i][0].split('T')
      if ('+' in listeIn[i][0][1]):
        listeIn[i][0][1] = listeIn[i][0][1].split('+')
        del listeIn[i][0][1][1]
    else :
      listeIn[i][0] = listeIn[i][0].split()

    if ('.' in listeIn[i][0][1]) : 
      listeIn[i][0] = listeIn[i][0][0].split('-') + listeIn[i][0][1].split('.')

    elif (isinstance(listeIn[i][0][1], list)):
      listeIn[i][0] = listeIn[i][0][0].split('-') + listeIn[i][0][1]
    else :
      listeIn[i][0][0] = listeIn[i][0][0].split('-')
  
    if (len(listeIn[i][0]) == 5):
      del listeIn[i][0][4]

    if (len(listeIn[i][0]) > 2):
      listeIn[i][0][3] = listeIn[i][0][3].split(':')
      listeIn[i][0] = listeIn[i][0][0:3] + listeIn[i][0][3]
    else :
      listeIn[i][0][1] = listeIn[i][0][1].split(':')
      listeIn[i][0] = listeIn[i][0][0] + listeIn[i][0][1]

    for j in range(6):
      listeIn[i][0][j] = int(listeIn[i][0][j])

    listeIn[i].insert(0,listeIn[i][0][0:3])
    listeIn[i].insert(1,listeIn[i][1][3:])
    del listeIn[i][2]
  
  return "Traitement réalisé avec succès"

In [745]:
convert_date(list_DataMeteo2020)

'Traitement réalisé avec succès'

In [746]:
# Utilisation de Astral pour obtenir : l'heure du levé de soleil, du coucher
# Ainsi que le nombre d'heures d'ensoleillement.
# A partir de ces données nous pourront exclures les données non pertinentes
# (heures de non ensoleillement) des data de "list_DataMeteo2020"

# Il existe néanmoins des méthodes plus complexes et plus précises
# de l'ordre de la minute pour calculer le nombre effectif d'heures 
# d'ensoleillement (cf angle terrestre etc...)
# https://www.esrl.noaa.gov/gmd/grad/solcalc/calcdetails.html 

city = LocationInfo('Gelos', 'France', 'Europe/London', 43.232951, 0.078082)

sunhours = []
mois_longs = [1,3,5,7,8,10,12]

# Réupère les horaires sur un an (en 2020) des couchés et levés de soleil
# La sortie est une liste de 365 elements de 2 listes
# (une liste pour le lever et une autre pour le coucher)
def getSunSetRise():
  for i in range(1,13):
    if (i in mois_longs) :
      for j in range(1,32):
        elem = daylight(city.observer,date=dt.date(2020,i,j),tzinfo=city.timezone)
        sunhours.append(convert_dataSun(elem))
    elif (i == 2) :
      for j in range(1,29):
        elem = daylight(city.observer,date=dt.date(2020,i,j),tzinfo=city.timezone)
        sunhours.append(convert_dataSun(elem))
    elif ((i not in mois_longs) and i!=2):
      for j in range(1,31):
        elem = daylight(city.observer,date=dt.date(2020,i,j),tzinfo=city.timezone)
        convert_dataSun(elem)
        sunhours.append(convert_dataSun(elem))
        
#Transforme la sortie d'astral (datetime.datetime) en liste d'int
def convert_dataSun(listeIn):
  res = []
  for i in range(len(listeIn)):
      tmp = str(listeIn[i])
      tmp = tmp.split('.')
      tmp = tmp[0].split(' ')
      tmp[0] = tmp[0].split('-')
      tmp[1] = tmp[1].split(':')
      res.append(tmp[0])
      res.append(tmp[1])
      for j in range(len(res)):
        for k in range(len(res[j])):
          res[j][k] = int(res[j][k])
  del res[2]
  return res

In [747]:
getSunSetRise()

In [748]:
#Convertit le jour (int) en une date [yy,mm,dd] (list)
def date_converter(daynumber):
  year = str(list_DataMeteo2020[0][0][0])
  day_num = str(daynumber)
  if (day_num == '0' or day_num== '-1'):
    res = [0,0,0]
    return res
  else :
    res = datetime.strptime(year + "-" +day_num, "%Y-%j").strftime("%m-%d-%Y")
    res = res.split('-')
    for i in range(len(res)):
      res[i] = int(res[i])
    res.reverse()
    res[1],res[2] = res[2],res[1]
    return res

#Récupère les data du jour en fonction du jour t (On récupère uniquement les données du jour !)
def get_dataOfTheDay(t):
  date_formated = date_converter(t)
  sun_rise = sunhours[t][1]
  sun_set = sunhours[t][2]
  resultat = []

  for i in range(len(list_DataMeteo2020)):
    if ((list_DataMeteo2020[i][0][1] == date_formated[1]) and (list_DataMeteo2020[i][0][2] == date_formated[2])):
      if ((sun_rise[0] <= list_DataMeteo2020[i][1][0]) and (sun_set[0] >= list_DataMeteo2020[i][1][0])):
        resultat.append(list_DataMeteo2020[i])
  return resultat

### **FACTEUR SAISON (2.1)**


In [749]:
#--------------------------------------------------------------#
@lru_cache(maxsize = None)
def f(t):
    return(1-1/(1+x1*np.exp(-2*t/x2)))

#--------------------------------------------------------------#
@lru_cache(maxsize = None)
def g(t):
    return(1/(1+x3*np.exp(-2*(t-x4)/x5))) 

#--------------------------------------------------------------#  

#Facteur saison (t en jours [max 365 jours])
@lru_cache(maxsize = None)
def season(t):
  print(max(g(t),f(t)))
  return max(g(t),f(t))

#--------------------------------------------------------------#

In [750]:
#--------------------------------------------------------------#
@lru_cache(maxsize = None)
def f(t):
    return(1-1/(1+x1*np.exp(-2*t/x2)))

#--------------------------------------------------------------#
@lru_cache(maxsize = None)
def g(t):
    return(1/(1+x3*np.exp(-2*(t-x4)/x5))) 

#--------------------------------------------------------------#  

#Facteur saison (t en jours [max 365 jours])
@lru_cache(maxsize = None)
def season(t):
    return max(g(t),f(t))

#--------------------------------------------------------------#

### **CELLULES DANS LA RUCHE ET TAUX DE PONTE (2.1)**

In [751]:

#--------------------------------------------------------------#

# Nombre de cellules disponibles (vides)
@lru_cache(maxsize = None)
def CELLSempty(t) :
    if (t<=0):
        return CELLShive
    else :
        return int(CELLShive - CELLSbrood(t)-STORESpollen(t)-STORESnectar(t)-STOREShoney(t))

#--------------------------------------------------------------#

# Fonction décrivant la diminution du taux de ponte lorsque le nombre de cellules disponibles devient limité 
@lru_cache(maxsize = None)
def SUPcomb(t):
    res = CELLSempty(t)/(CELLShive+1)
    if (res < SUPtreshold):
        return res / SUPtreshold
    else :
        return 1  

#--------------------------------------------------------------#

# Facteur stochastique (aléatoire) du taux de ponte
@lru_cache(maxsize = None)
def ELRstoch():
    return rd.uniform(-ELRstochrange,ELRstochrange)

#--------------------------------------------------------------#

# Fonction prédictive du taux de ponte de la reine
@lru_cache(maxsize = None) 
def ELR(t):
    return ELRbase*(1+ELRstoch())*(1-season(t))*SUPcomb(t)  

#--------------------------------------------------------------#    

### **OEUFS DANS LA RUCHE (2.2)**

In [752]:
#--------------------------------------------------------------#    

# Nombre d'oeuf au stade i à un instant t
@lru_cache(maxsize = None)
def EGGS(i,t):
    if (t<=0) :
        return 0
    # i désigne l'âge de l'individu immature, t le jour
    if i==1 :
        return ELR(t-1) * (1-MORTALITYeggs)
    elif i==2 or i==3 :
        return EGGS(i-1,t-1)*(1-MORTALITYeggs)
    else :
        # Exception, la fonction EGGS n'est pas définie pour i qui n'appartient pas à [|1;3|]
        print('Erreur, l age i n est pas possible pour la fonction EGGS daprès le modele')
        return (-1)

#--------------------------------------------------------------#    

# Nombre de cellules contenant des oeufs dans la ruche à un instant t
@lru_cache(maxsize = None) 
def CELLSeggs(t):
  # Nombre total d'oeuf dans la ruche à un instant t 
  return int(sum([EGGS(i,t) for i in range(1,LIFESPANegg+1)]))

#--------------------------------------------------------------#   

### **LARVES DANS LA RUCHE (2.2)**

In [753]:
#--------------------------------------------------------------#

# Taux de cannibalisme des larves
@lru_cache(maxsize = None)
def CANNIBALISMlarvae(i,t):
    return(CANNIBALISMbase[i-1]*(1-(INDEXpollensituation(t-1)*INDEXnursingquality(t-1))))

#--------------------------------------------------------------#  

# Taux de larves survivantes
@lru_cache(maxsize = None)
def SURVIVALlarvae(i,t):
    return((1-CANNIBALISMlarvae(i,t))*(1-MORTALITYlarvae))

#--------------------------------------------------------------#  

# Nombre de larves au stade i à un instant t
@lru_cache(maxsize = None)
def LARVAE(i,t):
    if (t<=0) :
        return 0
  # i désigne l'âge de l'individu immature, t le jour considéré
    if i==1 :
        return EGGS(LIFESPANegg,t-1)*SURVIVALlarvae(1,t)
    elif i>1 and i<=LIFESPANlarvae :
        return LARVAE(i-1,t-1)*SURVIVALlarvae(i,t)
    else :
        # Exception, la fonction LARVAE n'est pas définie pour i qui n'appartient pas à [|1;LIFESPANlarvae|]
        print('Erreur, l age i n est pas possible pour la fonction LARVAE daprès le modele')
        return (-1)

#--------------------------------------------------------------#

# Nombre de cellules contenant des larves dans la ruche à un instant t
@lru_cache(maxsize = None) 
def CELLSlarvae(t):
  # Nombre total de larves dans la ruche à un instant t  
  return int(sum([LARVAE(i,t) for i in range(1,LIFESPANlarvae+1)]))

#--------------------------------------------------------------#

### **PUPES DANS LA RUCHE (2.2)**

In [754]:
#--------------------------------------------------------------#

# Nombre de pupes au stade i à un instant t
@lru_cache(maxsize = None)
def PUPAE(i,t):
    if (t<=0) :
        return 0
    # i désigne l'âge de l'individu immature, t le jour
    if i==1 :
        return int(LARVAE(LIFESPANlarvae,t-1)*(1-MORTALITYpupae))
    elif i>1 and i<=LIFESPANpupae :
        return int(PUPAE(i-1,t-1)*(1-MORTALITYpupae))
    else :
        # Exception, la fonction PUPAE n'est pas définie pour i qui n'appartient pas à [|1;LIFESPANpupae|]
        print('Erreur, l age i n est pas possible pour la fonction PUPAE daprès le modele')
        return (-1)

#--------------------------------------------------------------#

# Nombre de cellules contenant des pupes dans la ruche à un instant t 
@lru_cache(maxsize = None)
def CELLSpupae(t):
  # Nombre total de pupes dans la ruche à un instant t  
  return int(sum([PUPAE(i,t) for i in range(1,LIFESPANpupae+1)]))

#--------------------------------------------------------------#

### **CELLULES DU COUVAIN DANS LA RUCHE (2.2)**

In [755]:
#--------------------------------------------------------------#
@lru_cache(maxsize = None)
def CELLSbrood(t):
  # Nombre d'individus en phase de développement dans la ruche (oeufs+larves+pupes)
  return int(CELLSeggs(t) + CELLSlarvae(t) + CELLSpupae(t))
#--------------------------------------------------------------#  

### **MORTALITE DES ABEILLES ADULTES (2.3)**

In [756]:
#--------------------------------------------------------------#

# Taux de mortalité des abeilles adultes à un instant t
@lru_cache(maxsize = None)
def MORTALITYadult(t):
    return MORTALITYadultbase + MORTALITYnursing*(NURSES(t-1)/(BEESadult(t-1)+1))+MORTALITYprocessing*(PROCESSORS(t-1)/(BEESadult(t-1)+1))+MORTALITYforaging*(FORAGERSactive(t-1)/(BEESadult(t-1)+1))

#--------------------------------------------------------------#

# Nombre d'abeilles adultes à un instant t
#@lru_cache(maxsize = None)
def BEESadult(t):
    if (t<=0) :
        return BEESadultINIT
    else :
        return int((BEESadult(t-1)+PUPAE(LIFESPANpupae,t-1))*(1-MORTALITYadult(t)))

#--------------------------------------------------------------#

### **INFLUENCE DE LA TEMPERATURE ET DU CLIMAT SUR LE BUTINAGE (2.4)**

Le principe théorique appliqué à la partie ci-dessous est :

*   Récupérer les données du jour concerné (sans celles de la nuit)
*   Convertir les données des variables (dépendamment de la nature et du processus de la fonction) en un index.
*   Ajouter la valeur normalisée (l'index) dans une liste
*   Retourner un index moyen de cette liste d'index




In [757]:
#--------------------------------------------------------------#
# Permet de définir si une année est bissextile (non implémenté dans le modèle)
def estBissextile(year) :
  bool_biss = False
  if (year%4==0 and year%100!=0 or year%400==0):
    bool_biss = True
  else : 
    print(year, "est une année non bissextile")
  return bool_biss

#--------------------------------------------------------------#  
# Donne la température pour le jour t
# On utilise uniquement les données des témpératures comprises entre 
# L'heure ou le soleil se lève et se couche
@lru_cache(maxsize = None) 
def TEMP(t):
  liste_data = get_dataOfTheDay(t)
  liste_data_temp = []
  #Récupère les data de température uniquement et retourne leur liste
  for i in range(len(liste_data)):
    liste_data_temp.append(liste_data[i][2])
  return liste_data_temp
#--------------------------------------------------------------#  
# Donne la durée (en heures) pednant laquelle il pleut lorsqu'il fait jour 
@lru_cache(maxsize = None)
def HOURSraining_during_daylight(t):
  liste_data = get_dataOfTheDay(t)
  liste_data_rain = []
  hours_Day_Light = HOURSdaylight(t)
  #Récupère les data de pluie uniquement et retourne leur liste
  for i in range(len(liste_data)):
    liste_data_rain.append(liste_data[i][3])
  nb_hours_rain = []
  precipitation = liste_data_rain
  for i in range(len(precipitation)):
    value = precipitation[i]
    if (value != 0) : 
      nb_hours_rain.append(value)
  return len(nb_hours_rain)

 
# Donne la durée (en heures) pendant laquelle il fait jour
# (On pourrait faire une soustraction des heures de coucher et lever du soleil)
# Mais le nombre de données par heure n'est pas toujours égal au nombre d'heures
# d'ensoleillement.
@lru_cache(maxsize = None)
def HOURSdaylight(t):
  liste_data = get_dataOfTheDay(t)
  return len(liste_data)

#--------------------------------------------------------------#
# Taux de pluie lorsqu'il fait jour
@lru_cache(maxsize = None)
def RAIN(t):
    if (t<=0):
        return 0
    else:
        if (HOURSdaylight(t)==0):
          return HOURSraining_during_daylight(t)
        else :
          return (HOURSraining_during_daylight(t)/HOURSdaylight(t))

#--------------------------------------------------------------#
# Proche de 1 -> la pluie n'a pas empêché les abeilles d'aller butiner au jour t. 
# Proche de 0 -> la pluie a empêché les abeilles d'aller butiner pendant la quasi totalité de la journée.
@lru_cache(maxsize = None)
def INDEXrain(t):
    if ((1-RAIN(t)) < 0) :
      return 0
    else :
      return 1-RAIN(t)

#--------------------------------------------------------------#
# Proche de 1 -> la température était favorable au butinage au jour t. 
# Proche de 0 -> la température a empêché les abeilles d'aller butiner pendant la quasi totalité de la journée.
@lru_cache(maxsize = None)
def INDEXtemperature(t):
    if (t<= 0):
        return 0
    else:
        temp = TEMP(t)
        index_res = []
        for i in range(len(temp)):
          value = temp[i]
          if (value <= 14):
            index_res.append(0)
          if (value>14 and value<=22):
            res = (value - 14)/8
            index_res.append(res)
          if (value>22 and value<=32):
            index_res.append(1)
          if (value>32 and value<=40):
            res = (40-value)/8
            index_res.append(res)
          if (value>40):
            index_res.append(0)
    if len(temp)==0:
      return 0
    else :
      return sum(index_res)/len(temp)
#--------------------------------------------------------------# 
# Index du vent en fonction de sa vitesse
# 0 les abeilles ne sortent pas (vent > 45 km/h)
# 1 les abeilles sortent (vent < 45km/h)
def INDEXwind(t):
  liste_data = get_dataOfTheDay(t)
  liste_data_wind = []
  #Récupère les data de vent uniquement et retourne leur liste
  for i in range(len(liste_data)):
    liste_data_wind.append(liste_data[i][5])

  wind = liste_data_wind
  index_res = []
  for i in range(len(wind)):
    value = wind[i]
    if (value>=0 and value<=max_wind):
      index_res.append(1-(value/max_wind))
    else :
      index_res.append(0)
  if len(wind) == 0:
    return 0
  else :
    return sum(index_res)/len(wind)
#--------------------------------------------------------------#  
# Proche de 1 -> Les conditions climatiques étaient favorables au butinage. 
# Proche de 0 -> Les conditions climatiques ont empêché les abeilles d'aller butiner pendant la quasi totalité de la journée.
@lru_cache(maxsize = None)
def INDEXflight(t):
    return INDEXrain(t) * INDEXtemperature(t) * INDEXwind(t)

#--------------------------------------------------------------# 
#Ajout de la contrainte d'humidité (<60% pas de nectar)
@lru_cache(maxsize = None)
def INDEXnectaroutside(t):
  liste_data = get_dataOfTheDay(t)
  liste_data_humidity = []
  #Récupère les data d'humidité uniquement et retourne leur liste
  for i in range(len(liste_data)):
    liste_data_humidity.append(liste_data[i][4])
    
  humidity = liste_data_humidity
  index_res = []
  for i in range(len(humidity)):
    value = humidity[i]
    if (value<min_humidity):
      index_res.append(0)
    else :
      index_res.append(min((1-season(t))*1.5,1))
  if len(humidity) == 0:
    return 0
  else :
    return sum(index_res)/len(humidity)

#--------------------------------------------------------------# 
@lru_cache(maxsize = None)
def INDEXpollenoutside(t):
    return min((1-season(t))*1.5,1)

#--------------------------------------------------------------#

### **DECISION ET REGULATION DES TACHES DE TRAVAIL (2.5 et 2.6)**

In [758]:
#--------------------------------------------------------------#

# Modelise le besoin journalier en abeilles travailleuses
@lru_cache(maxsize = None)
def NEEDworkers(t):
    return NEEDnurses(t)+NEEDpollenforagers(t)

#--------------------------------------------------------------#

# calcul le ratio des abeilles disponibles pour le travail
# si ratio < 1 alors (1-RATIOworkforce(t)) du travail necessaire ne sera pas effectué
@lru_cache(maxsize = None)
def RATIOworkforce(t):
    ratio=(BEESadult(t)*(1-FACTORothertasks))/(NEEDworkers(t)+1)
    if ratio<1:
        return ratio
    else:
        return 1

#--------------------------------------------------------------#

# Somme toute les demandes nouricieres de chaque couvins
@lru_cache(maxsize = None)
def NEEDnurses(t):
    total=0
    for i in range(1,LIFESPANlarvae+1):
        total+=LARVAE(i,t)*NEEDnurses_per_larva[i-1]
    return int(total+(CELLSeggs(t)*NEEDnurses_per_egg)+(CELLSpupae(t)*NEEDnurses_per_pupa))

#--------------------------------------------------------------#

# Nombre d'abeilles nouricieres actives
@lru_cache(maxsize = None)
def NURSES(t):
    return (NEEDnurses(t)*RATIOworkforce(t))

#--------------------------------------------------------------#

# Permet de modeliser une importante boucle de retour
@lru_cache(maxsize = None)
def INDEXnursingquality(t):
    return NURSES(t)/(NEEDnurses(t)+1)

#--------------------------------------------------------------#

### **BUTINAGE ET BESOINS EN RESSOURCES (2.7)**

In [759]:
#--------------------------------------------------------------#

# Calcule le nombre de butineuses potentielles
@lru_cache(maxsize = None)
def FORAGERS(t):
	return int(FORAGERSpollen(t) + FORAGERSnectar(t))

#--------------------------------------------------------------#

# Calcule le nombre de butineuses qui sortent de la ruche
@lru_cache(maxsize = None)
def FORAGERSactive(t):
	return (FORAGERSpollenactive(t) + FORAGERSnectaractive(t))

#--------------------------------------------------------------#

# Nombre d'abeilles butineuses actives
@lru_cache(maxsize = None)
def FORAGERSnectaractive(t):
    return FORAGERSnectar(t)*INDEXflight(t)*INDEXnectaroutside(t)

#--------------------------------------------------------------#

# Nombre d'abeilles disponibles pour aller recolter du nectar
@lru_cache(maxsize = None)
def FORAGERSnectar(t):
    return min(BEESadult(t)*FACTORforagingmax-FORAGERSpollen(t),WORKFORCEnectar(t)-PROCESSORS(t))

#--------------------------------------------------------------#

# Besoin en pollen total
@lru_cache(maxsize = None)
def NEEDpollen(t):
    if (t<=0):
        return 0
    else:
        return NEEDpollen_larvae(t)+NEEDpollen_adult(t)

#--------------------------------------------------------------#

# Calcule le besoin en pollen de la part des larves
@lru_cache(maxsize = None)
def NEEDpollen_larvae(t):
    res = 0
    for i in range(1,LIFESPANlarvae+1):
        res = res + POLLENNEEDlarva[i-1]*LARVAE(i,t)
    return res

#--------------------------------------------------------------#

# Calcule le besoin en pollen de la part des abeilles adultes
@lru_cache(maxsize = None)
def NEEDpollen_adult(t):
    return BEESadult(t)*POLLENNEEDadult + NURSES(t)*POLLENNEEDnurse

#--------------------------------------------------------------#

# Calcule le besoin journalier en pollen à recolter basé sur les reserves moyenées entre j0 et j-2
@lru_cache(maxsize = None)
def NEEDpollenincome(t):
    acc = 0
    for d in range(3):
        acc = acc + NEEDpollen(t-d)
    acc = acc/3 * FACTORpollenstorage - STORESpollen(t)
    return max(0, acc)

#--------------------------------------------------------------#

# Calcule le nombre de butineuses de pollen requis
@lru_cache(maxsize = None)
def NEEDpollenforagers(t):
    return (NEEDpollenincome(t-1) / (LOADpollenforager*TURNSpollenforager*FACTORforagingsuccess))

#--------------------------------------------------------------#

# Calcule le nombre potentiel de butineuses de pollen
@lru_cache(maxsize = None)
def FORAGERSpollen(t):
    maximum = max(NEEDpollenforagers(t)*RATIOworkforce(t), (BEESadult(t)-NURSES(t))*FACTORminpollenforagers )
    return min(maximum, BEESadult(t)*FACTORforagingmax)

#--------------------------------------------------------------#

# Calcule le nombre de butineuses de pollen qui sortent de la ruche
@lru_cache(maxsize = None)
def FORAGERSpollenactive(t):
    return FORAGERSpollen(t)*INDEXflight(t)*INDEXpollenoutside(t)
 

#--------------------------------------------------------------#

# Calcule le besoin journalier en nectar de la ruche
@lru_cache(maxsize = None)
def NEEDnectar(t):
    return NEEDnectar_larvae(t)+NEEDnectar_adult(t)
#--------------------------------------------------------------#

# Calcule le besoin en nectar de la part des larves
@lru_cache(maxsize = None)
def NEEDnectar_larvae(t):
    acc = 0
    for i in range(1,LIFESPANlarvae+1):
        acc = acc + (NECTARNEEDlarva[i-1]*LARVAE(i,t))
    return acc

#--------------------------------------------------------------#

# Calcule la demande en nectar de la part des abeilles adultes
@lru_cache(maxsize = None)
def NEEDnectar_adult(t):
    # nectar necessaire pour les abeilles adultes
    nectar_adults = BEESadult(t)*NECTARNEEDadult

    # nectar necessaire pour les abeilles nouricieres
    nectar_nurses = NURSES(t)*NECTARNEEDnurse

    # nectar necessaire pour les butineuses actives
    nectar_foragers = FORAGERSactive(t)*NECTARNEEDactiveforager

    # Retourne la demande en nectar des abeilles adultes
    return nectar_adults+nectar_nurses+nectar_foragers

#--------------------------------------------------------------#

# Calcule la force de travail restante pour le nectar
@lru_cache(maxsize = None)
def WORKFORCEnectar(t):
    if RATIOworkforce(t)==1:
        res = BEESadult(t)*(1-FACTORothertasks)
        return res-NURSES(t)-FORAGERSpollen(t)
    else :
        return 0

#--------------------------------------------------------------#

### **FLUX DE RESSOURCES ET TRAITEMENT DE LA NOURRITURE AU SEIN DE LA COLONIE (2.8  et 2.9)**

In [760]:
#--------------------------------------------------------------#

# Permet de calculer le pollen produit chaque jour
@lru_cache(maxsize = None)
def INCOMEpollen(t):
    return FORAGERSpollenactive(t) * LOADpollenforager * TURNSpollenforager * FACTORforagingstoch(t) * FACTORforagingsuccess

#--------------------------------------------------------------#

# Permet de calculer le facteur stochastique
@lru_cache(maxsize = None)
def FACTORforagingstoch(t):
    return(rd.uniform(1-stochasticForagingFactor , 1+stochasticForagingFactor))

#--------------------------------------------------------------#

# Decrit le niveau de pollen stocké par rapport aux besoins de la population
@lru_cache(maxsize = None)
def INDEXpollensituation(t):
    return min([1 , (STORESpollen(t)) / (NEEDpollen(t)*FACTORpollenstorage+1)])

#--------------------------------------------------------------#

# Décrit la production de nectar
@lru_cache(maxsize = None)
def INCOMEnectar(t):
    return (min([FORAGERSnectaractive(t) * LOADnectarforager * TURNSnectarforager * FACTORforagingstoch(t) * FACTORforagingsuccess , CELLSempty(t-1)]))

#--------------------------------------------------------------#

# Determine les processeurs de nectar necessaires
@lru_cache(maxsize = None)
def NEEDprocessors(t):
    return STORESnectar(t-1) * ProcessorsPerCell

#--------------------------------------------------------------#

@lru_cache(maxsize = None)
def PROCESSORS(t):
    return (min([NEEDprocessors(t) , WORKFORCEnectar(t)]))

#--------------------------------------------------------------#

# Determine le nectar produit 
@lru_cache(maxsize = None)
def PROCESSEDnectar(t):
    return (min([STORESnectar(t-1) - USAGEnectar(t) , PROCESSORS(t) / ProcessorsPerCell]))

#--------------------------------------------------------------#

### **GESTION DES RESSOURCES ET TRAITEMENT (2.10)**

In [761]:
#--------------------------------------------------------------#

# Modelise l'utilisation du nectar dans la colonie
@lru_cache(maxsize = None)
def USAGEnectar(t):
    liste_var = []
    needNectar_var = NEEDnectar(t)
    storeNectar_var = STORESnectar(t-1)
    liste_var.append(needNectar_var)
    liste_var.append(storeNectar_var)
    return min(liste_var)

#--------------------------------------------------------------#

#Modelise l'utilisation du pollen dans la colonie
@lru_cache(maxsize = None)
def USAGEpollen(t):
    liste_var = []
    needPollen_var = NEEDpollen(t-1)*(1-(FACTORpollensavingmax*(1-INDEXpollensituation(t-1))))
    storePollen_var = STORESpollen(t-1)

    liste_var.append(needPollen_var)
    liste_var.append(storePollen_var)
    return min(liste_var)

#--------------------------------------------------------------#

# Modelise l'utilisation du miel dans la colonie
@lru_cache(maxsize = None)
def USAGEhoney(t):
    liste_var = []
    storeHoney_var = STOREShoney(t-1)
    nectar_var = (NEEDnectar(t)-USAGEnectar(t))*RATIOnectar_to_honey

    liste_var.append(storeHoney_var)
    liste_var.append(nectar_var)
    return min(liste_var)


#--------------------------------------------------------------#

# Modelise le stockage du pollen dans la colonie
@lru_cache(maxsize = None)
def STORESpollen(t):
    if (t<=0):
        return 0
    else:
        return STORESpollen(t-1)+INCOMEpollen(t)-USAGEpollen(t)

#--------------------------------------------------------------#

# Modelise le stockage du nectar dans la colonie
@lru_cache(maxsize = None)
def STORESnectar(t):
    if (t<=0):
        return 0
    else :
        return STORESnectar(t-1)+INCOMEnectar(t)-USAGEnectar(t)-PROCESSEDnectar(t)

#--------------------------------------------------------------#

# Modelise le stockage du miel dans la colonie
@lru_cache(maxsize = None)
def STOREShoney(t):
    if (t<=0):
        return STOREShoneyINIT
    else :  
        return STOREShoney(t-1)-USAGEhoney(t)+(PROCESSEDnectar(t)*RATIOnectar_to_honey)

#--------------------------------------------------------------#

# Modelise le poids de la colonie
@lru_cache(maxsize = None)
def WEIGHTcolony(t):
    somme_res = 0
    for i in range (1, LIFESPANlarvae):
        somme_res += w_larva[i-1]*LARVAE(i,t)
        return 1/1000 * (w_hivebase + (w_cellsbase * CELLShive) + (w_pollen * STORESpollen(t)) + (w_nectar * STORESnectar(t)) + (w_honey * STOREShoney(t)) + (w_egg * CELLSeggs(t)) + (w_pupa * CELLSpupae(t)) + somme_res + w_adult * BEESadult(t) )

#--------------------------------------------------------------#

# Modelise le nombre d'abeilles paresseuses dans la colonie
@lru_cache(maxsize = None)
def BEESlazy(t):
    return (BEESadult(t)*(1-FACTORothertasks))-FORAGERSactive(t)-NURSES(t)-PROCESSORS(t)

#--------------------------------------------------------------#

### **TEST DES FONCTIONS**

## **PARTIE LPO**

### **TRAITEMENT LPO**

In [762]:
# Définition de la fonction HoPoMo liant toutes les fonctions du modèle précédent.
def HOPOMO(nbdays):
    date = []
    seasonVar = []
    beesadult = []
    beeslazy = []
    cellsbrood = []
    cellseggs = []
    cellsempty = []
    cellslarvae = []
    cellspupae = []
    elr = []
    foragers = []
    foragersactive = []
    foragersnectar = []
    foragersnectaractive = []
    foragersnectaractiveTEST = []
    foragerspollen = []
    foragerspollenactive = []
    incomenectar = []
    incomepollen = []
    indexflight = []
    indexrain = []
    indexvent = []
    # indexhumidity = []
    indextemperature = []
    indexnectaroutside = []
    indexnursingquality = []
    indexpollenoutside = []
    indexpollensituation = []
    larvae1 = []
    larvae2 = []
    larvae3 = []
    larvae4 = []
    larvae5 = []
    mortalityadult = []
    neednectar = []
    neednectar_adult = []
    neednectar_larvae = []
    neednurses = []
    needpollen = []
    needpollen_adult = []
    needpollen_larvae = []
    needpollenforagers = []
    needpollenincome = []
    needprocessors = []
    needworkers = []
    nurses = []
    processednectar = []
    processors = []
    ratioworkforce = []
    storeshoney = []
    storesnectar = []
    storespollen = []
    supcomb = []
    #temp = np.concatenate([temp1,temp2])
    #temp1 = np.linspace(15, 35, 183)
    #temp2 = np.linspace(35, 15, 183)
    usagehoney = []
    usagenectar = []
    usagepollen = []
    weightcolony = []
    workforcenectar = []

    t = np.arange(nbdays+1)
    # nb_days est défini dans les paramètres à 364 lorsque l'on veut tester
    # le modèle sur toute une année (2020 ici).
    # Ce paramètre est redéfini dans la partie prévision (voir IMPORT DES DONNEES DE PREVISION)
    for i in range(nb_days):
        date.append(date_converter(i))
        seasonVar.append(season(i))
        beesadult.append(BEESadult(i))
        beeslazy.append(BEESlazy(i))
        cellsbrood.append(CELLSbrood(i))
        cellseggs.append(CELLSeggs(i))
        cellsempty.append(CELLSempty(i))
        cellslarvae.append(CELLSlarvae(i))
        cellspupae.append(CELLSpupae(i))
        elr.append(ELR(i))
        foragers.append(FORAGERS(i))
        foragersactive.append(FORAGERSactive(i))
        foragersnectar.append(FORAGERSnectar(i))
        foragersnectaractive.append(FORAGERSnectaractive(i))
        foragerspollen.append(FORAGERSpollen(i))
        foragerspollenactive.append(FORAGERSpollenactive(i))
        incomenectar.append(INCOMEnectar(i))
        incomepollen.append(INCOMEpollen(i))
        indexflight.append(INDEXflight(i))        
        indexrain.append(INDEXrain(i))
        indexvent.append(INDEXwind(i))
        # indexhumidity.append(INDEXhumidity(i))
        indextemperature.append(INDEXtemperature(i))
        indexnectaroutside.append(INDEXnectaroutside(i))
        indexnursingquality.append(INDEXnursingquality(i))
        indexpollenoutside.append(INDEXpollenoutside(i))
        indexpollensituation.append(INDEXpollensituation(i))
        larvae1.append(LARVAE(1,i))
        larvae2.append(LARVAE(2,i))
        larvae3.append(LARVAE(3,i))
        larvae4.append(LARVAE(4,i))
        larvae5.append(LARVAE(5,i))
        mortalityadult.append(MORTALITYadult(i))
        neednectar_adult.append(NEEDnectar_adult(i))
        neednectar_larvae.append(NEEDnectar_larvae(i))
        neednectar.append(NEEDnectar(i))
        neednurses.append(NEEDnurses(i))
        needpollen_adult.append(NEEDpollen_adult(i))
        needpollen_larvae.append(NEEDpollen_larvae(i))
        needpollen.append(NEEDpollen(i))
        needpollenforagers.append(NEEDpollenforagers(i))
        needpollenincome.append(NEEDpollenincome(i))
        needprocessors.append(NEEDprocessors(i))
        needworkers.append(NEEDworkers(i))
        nurses.append(NURSES(i))
        processednectar.append(PROCESSEDnectar(i))
        processors.append(PROCESSORS(i))
        ratioworkforce.append(RATIOworkforce(i))
        storeshoney.append(STOREShoney(i))
        storesnectar.append(STORESnectar(i))
        storespollen.append(STORESpollen(i))
        supcomb.append(SUPcomb(i))
        usagehoney.append(USAGEhoney(i))
        usagenectar.append(USAGEnectar(i))
        usagepollen.append(USAGEpollen(i))
        weightcolony.append(WEIGHTcolony(i))
        workforcenectar.append(WORKFORCEnectar(i))
    hopomo = pd.DataFrame({
        'date' : date,
        'seasonVar' : seasonVar,
        'beesadult' : beesadult,
        'beeslazy' : beeslazy,
        'cellsbrood' : cellsbrood,
        'cellseggs' : cellseggs,
        'cellsempty' : cellsempty,
        'cellslarvae' : cellslarvae,
        'cellspupae' : cellspupae,
        'elr' : elr,
        'foragers' : foragers,
        'foragersactive' : foragersactive,
        'foragersnectar' : foragersnectar,
        'foragersnectaractive' : foragersnectaractive,
        'foragerspollen' : foragerspollen,
        'foragerspollenactive' : foragerspollenactive,
        'incomenectar' : incomenectar,
        'incomepollen' : incomepollen,
        'indexflight' : indexflight,        
        'indexrain' : indexrain,
        'indexvent' : indexvent, 
        # 'indexhumidity' : indexhumidity,       
        'indextemperature' : indextemperature,
        'indexnectaroutside' : indexnectaroutside,
        'indexnursingquality' : indexnursingquality,
        'indexpollenoutside' : indexpollenoutside,
        'indexpollensituation' : indexpollensituation,
        'larvae1' : larvae1,
        'larvae2' : larvae2,
        'larvae3' : larvae3,
        'larvae4' : larvae4,
        'larvae5' : larvae5,
        'mortalityadult' : mortalityadult,
        'neednectar_adult' : neednectar_adult,
        'neednectar_larvae' : neednectar_larvae,
        'neednectar' : neednectar,
        'neednurses' : neednurses,
        'needpollen_adult' : needpollen_adult,
        'needpollen_larvae' : needpollen_larvae,
        'needpollen' : needpollen,
        'needpollenforagers' : needpollenforagers,
        'needpollenincome' : needpollenincome,
        'needprocessors' : needprocessors,
        'needworkers' : needworkers,
        'nurses' : nurses,
        'processednectar' : processednectar,
        'processors' : processors,
        'ratioworkforce' : ratioworkforce,
        'storeshoney' : storeshoney,
        'storesnectar' : storesnectar,
        'storespollen' : storespollen,
        'supcomb' : supcomb,
        'usagehoney' : usagehoney,
        'usagenectar' : usagenectar,
        'usagepollen' : usagepollen,
        'weightcolony' : weightcolony,
        'workforcenectar' : workforcenectar,
                                })
    return hopomo

In [763]:
# On lance le modèle
hopomo = HOPOMO(nb_days)

In [ ]:
# On sauvegarde les résultats des fonctions dans un csv avec la date sous forme de liste pour une meilleur compréhension du fichier
hopomo.to_csv("hopomo.csv")
hopomo.info()

### **GRAPHES ET COMPARAISON**

1. Comme proposé, nous comparons les courbes des couvains des ruches **R1** et **R6** situées à **Gelos** en **2020** avec notre courbe de couvain de la même année.
Il est nécessaire pour réaliser cette comparaison de passer nos valeurs en pourcentage selon l'explication donnée suivante :

  Le pourcentage de couvain peut être corrélé avec le nombre de cadres d'un couvain : 10% = 1 cadre, 80% = 8 cadres, etc

  Dans la ruche il y a 10 cadres qui ont de chaque côté 4500 cellules ce qui dans le corps de 10 cadres fait : 4500x20 = 90,000 cellules

  Puisque le couvain ne peut pas être à 100% partout (il faut de la place pour le nectar, du pollen etc) on prend un coef de remplissage de 50% et on finit par trouver que :

  **Nb_Cells_Couvain_reel = couvain_mesure(%) * 0,5 * 90000**

  Néanmoins, afin d'obtenir des courbes de même grandeur, le nombre de cellules a été adapté pour concorder au mieux avec les paramètres inconnus utilisés par R1 et R6.


2. Nous affichons les **graphes principaux** résultant du traitement LPO, et celui de la comparaison des couvains (%)

3. Nous affichons les **graphes secondaires** représentant d'autres variables telles que les stocks ou encore les index météorologiques.

#### **1. COMPARAISON DES COUVAINS** 

In [765]:
#Comparatif des couvain en pourcentage

#Fonction de récupération des valeurs du couvain et transformation en pourcentages
def getBroodValues():
  varBrood = hopomo.cellsbrood
  resBrood = []
  max_cell = 100*0.5*435 #cf formule ci-dessus
  print(max_cell)
  for i in range(len(varBrood)):
    resBrood.append((varBrood[i] * 100)/ max_cell )
  return resBrood

#Chargement des données .txt des ruches R1 et R6
evolCouvR1 = pd.read_csv("/content/R1_couvain.txt",sep="\t")
evolCouvR2 = pd.read_csv("/content/R6_couvain.txt",sep="\t")

#Récupération des valeurs en pourcentage qui nous intérressent
dataR1 = pd.DataFrame(evolCouvR1,columns=['pourc'])
dataR2 = pd.DataFrame(evolCouvR2,columns=['pourc'])

#Conversion en listes des données des couvains de R1 et R6
list_DataevolCouvR1 = [list(row) for row in dataR1.values]
dataListR1 = [item for sublist in list_DataevolCouvR1 for item in sublist]

list_DataevolCouvR2 = [list(row) for row in dataR2.values]
dataListR2 = [item for sublist in list_DataevolCouvR2 for item in sublist]

#### **2. GRAPHES PRINCIPAUX**

In [766]:
fig = make_subplots(rows=3, cols=2, vertical_spacing=0.05)

fig.add_trace(
    go.Scatter(y=hopomo.beesadult, name="Adult Bees", mode='lines', xaxis="x", yaxis="y1", showlegend = True),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(y=hopomo.storespollen, name="Pollen Stores", mode='lines', xaxis="x", yaxis="y2",showlegend = True),
    row=1, col=2
)

#-------------------------------------------------------------------------------------------------------------------------------------------------------------------#
fig.add_trace(
    go.Scatter(y=hopomo.cellsbrood, name="Brood cells", mode='lines', xaxis="x", yaxis="y3",showlegend = True),
    row=2, col=1
)

#-------------------------------------------------------------------------------------------------------------------------------------------------------------------#

fig.add_trace(
    go.Scatter(y=getBroodValues(), name="Brood cells in %", mode='lines', xaxis="x", yaxis="y3",showlegend = True),
    row=3, col=1
)

fig.add_trace(
    go.Scatter(y=dataListR1, name="R1", mode='lines', xaxis="x", yaxis="y3",showlegend = True),
    row=3, col=1
)

fig.add_trace(
    go.Scatter(y=dataListR2, name="R6", mode='lines', xaxis="x", yaxis="y3",showlegend = True),
    row=3, col=1
)

#-------------------------------------------------------------------------------------------------------------------------------------------------------------------#

fig.add_trace(
    go.Scatter(y=hopomo.storeshoney, name="Honey Stores", mode='lines', xaxis="x", yaxis="y4", showlegend = True),
    row=2, col=2
)

fig.add_trace(
    go.Scatter(y=hopomo.weightcolony, name="Colony Weight", mode='lines', xaxis="x", yaxis="y6",showlegend = True),
    row=3, col=2
)

fig.update_yaxes(title_text="adult bees", range=[0, 60000], row=1, col=1)
fig.update_yaxes(title_text="pollen stores", range=[0,6000], row=1, col=2)
fig.update_yaxes(title_text="brood cells", range=[0, 35000], row=2, col=1)
fig.update_yaxes(title_text="honey stores", range=[0, 100000], row=2, col=2)
fig.update_yaxes(title_text="colony weight (kg)", range=[0, 100], row=3, col=2)
fig.update_yaxes(title_text="brood % comparaison", range=[0, 100], row=3, col=1)
#fig.update_yaxes(title_text = 'Temp (°C)', ticks="outside", range=[-4, 30], tickwidth=1, nticks=30, tickcolor='crimson', showgrid=False, showline = True,secondary_y=False, spikesnap='cursor',spikethickness=1)
fig.update_traces(hovertemplate="day: %{x} <br>val: %{y:.1f}")
fig.update_layout(autosize=False, height=1200, width=1200)
fig.show()


21750.0


#### **3. AUTRES GRAPHES**

In [767]:
fig2 = make_subplots(rows=10, cols=1, vertical_spacing=0.005, shared_xaxes=True)

#Cells 2020
fig2.add_trace(go.Scatter(y=hopomo.storeshoney, name="storeshoney", mode='lines', xaxis="x", yaxis="y1",stackgroup='one'),
    row=1, col=1)
fig2.add_trace(go.Scatter(y=hopomo.storesnectar, name="storesnectar", mode='lines', xaxis="x", yaxis="y1",stackgroup='one'),
    row=1, col=1)
fig2.add_trace(go.Scatter(y=hopomo.storespollen, name="storespollen", mode='lines', xaxis="x", yaxis="y1",stackgroup='one'),
    row=1, col=1) 
fig2.add_trace(go.Scatter(y=hopomo.cellseggs, name="cellseggs", mode='lines', xaxis="x", yaxis="y1",stackgroup='one'),
    row=1, col=1)
fig2.add_trace(go.Scatter(y=hopomo.cellslarvae, name="cellslarvae", mode='lines', xaxis="x", yaxis="y1",stackgroup='one'),
    row=1, col=1)
fig2.add_trace(go.Scatter(y=hopomo.cellspupae, name="cellspupae", mode='lines', xaxis="x", yaxis="y1",stackgroup='one'),
    row=1, col=1)
fig2.add_trace(go.Scatter(y=hopomo.cellsbrood, name="cellsbrood", mode='lines', xaxis="x", yaxis="y1"),
    row=1, col=1)
fig2.add_trace(go.Scatter(y=hopomo.cellsempty, name="cellsempty", mode='lines', xaxis="x", yaxis="y1",stackgroup='one'),
    row=1, col=1)

#Income 2020
fig2.add_trace(go.Scatter(y=hopomo.incomenectar, name="incomenectar", mode='lines', xaxis="x", yaxis="y1",stackgroup='one'),
    row=2, col=1)
fig2.add_trace(go.Scatter(y=hopomo.incomepollen, name="incomepollen", mode='lines', xaxis="x", yaxis="y1",stackgroup='one'),
    row=2, col=1)

#Indexes 2020
fig2.add_trace(go.Scatter(y=hopomo.indexflight, name="indexflight", mode='lines', xaxis="x", yaxis="y1"),
    row=3, col=1)
fig2.add_trace(go.Scatter(y=hopomo.indexrain, name="indexrain", mode='lines', xaxis="x", yaxis="y1"),
    row=3, col=1)
fig2.add_trace(go.Scatter(y=hopomo.indextemperature, name="index temperature", mode='lines', xaxis="x", yaxis="y1"),
    row=3, col=1)
fig2.add_trace(go.Scatter(y=hopomo.indexvent, name="indexvent", mode='lines', xaxis="x", yaxis="y1"),
    row=3, col=1)
fig2.add_trace(go.Scatter(y=hopomo.seasonVar, name="index season", mode='lines', xaxis="x", yaxis="y1"),
    row=3, col=1)
# fig2.add_trace(go.Scatter(y=hopomo.indexhumidity, name="indexhumidity", mode='lines', xaxis="x", yaxis="y1"),
#     row=3, col=1)


# Index divers 2020
fig2.add_trace(go.Scatter(y=hopomo.indexnectaroutside, name="index nectaroutside", mode='lines', xaxis="x", yaxis="y1"),
    row=4, col=1)
fig2.add_trace(go.Scatter(y=hopomo.indexpollenoutside, name="index pollenoutside", mode='lines', xaxis="x", yaxis="y1"),
    row=4, col=1)
fig2.add_trace(go.Scatter(y=hopomo.indexpollensituation, name="index pollensituation", mode='lines', xaxis="x", yaxis="y1"),
    row=4, col=1)
fig2.add_trace(go.Scatter(y=hopomo.indexnursingquality, name="index nursingquality", mode='lines', xaxis="x", yaxis="y1"),
    row=4, col=1)

# Foragers 2020
fig2.add_trace(go.Scatter(y=hopomo.foragersactive, name="foragersactive", mode='lines', xaxis="x", yaxis="y1",fill='tozeroy'),
    row=5, col=1)
fig2.add_trace(go.Scatter(y=hopomo.foragersnectar, name="foragersnectar", mode='lines', xaxis="x", yaxis="y1"),
    row=5, col=1)
fig2.add_trace(go.Scatter(y=hopomo.foragersnectaractive, name="foragersnectar active", mode='lines', xaxis="x", yaxis="y1",fill='tozeroy'),
    row=5, col=1)
fig2.add_trace(go.Scatter(y=hopomo.foragerspollen, name="foragerspollen", mode='lines', xaxis="x", yaxis="y1"),
    row=5, col=1)
fig2.add_trace(go.Scatter(y=hopomo.foragerspollenactive, name="foragerspollen active", mode='lines', xaxis="x", yaxis="y1",fill='tozeroy'),
    row=5, col=1)

# Nurse and needs 2020
fig2.add_trace(go.Scatter(y=hopomo.nurses, name="nurses", mode='lines', xaxis="x", yaxis="y1"),
    row=6, col=1)
fig2.add_trace(go.Scatter(y=hopomo.neednurses, name="neednurses", mode='lines', xaxis="x", yaxis="y1"),
    row=6, col=1)
fig2.add_trace(go.Scatter(y=hopomo.needworkers, name="needworkers", mode='lines', xaxis="x", yaxis="y1"),
    row=6, col=1)
fig2.add_trace(go.Scatter(y=hopomo.needprocessors, name="needprocessors", mode='lines', xaxis="x", yaxis="y1"),
    row=6, col=1)
fig2.add_trace(go.Scatter(y=hopomo.needpollenforagers, name="needpollenforagers", mode='lines', xaxis="x", yaxis="y1"),
    row=6, col=1)

# Larvae 2020
fig2.add_trace(go.Scatter(y=hopomo.larvae1, name="larvae 1", mode='lines', xaxis="x", yaxis="y1"),
    row=7, col=1)
fig2.add_trace(go.Scatter(y=hopomo.larvae2, name="larvae 2", mode='lines', xaxis="x", yaxis="y1"),
    row=7, col=1)
fig2.add_trace(go.Scatter(y=hopomo.larvae3, name="larvae 3", mode='lines', xaxis="x", yaxis="y1"),
    row=7, col=1)
fig2.add_trace(go.Scatter(y=hopomo.larvae4, name="larvae 4", mode='lines', xaxis="x", yaxis="y1"),
    row=7, col=1)
fig2.add_trace(go.Scatter(y=hopomo.larvae5, name="larvae 5", mode='lines', xaxis="x", yaxis="y1"),
    row=7, col=1)

# Needs 2020
fig2.add_trace(go.Scatter(y=hopomo.needpollen, name="needpollen", mode='lines', xaxis="x", yaxis="y1"),
    row=8, col=1)
fig2.add_trace(go.Scatter(y=hopomo.needpollenincome, name="needpollen income", mode='lines', xaxis="x", yaxis="y1"),
    row=8, col=1)
fig2.add_trace(go.Scatter(y=hopomo.needpollen_adult, name="needpollen_adult", mode='lines', xaxis="x", yaxis="y1"),
    row=8, col=1)
fig2.add_trace(go.Scatter(y=hopomo.needpollen_larvae, name="needpollen_larvae", mode='lines', xaxis="x", yaxis="y1"),
    row=8, col=1)
fig2.add_trace(go.Scatter(y=hopomo.storespollen, name="storespollen", mode='lines', xaxis="x", yaxis="y1"),
    row=8, col=1)

# Needs nectar 2020
fig2.add_trace(go.Scatter(y=hopomo.neednectar, name="neednectar", mode='lines', xaxis="x", yaxis="y1"),
    row=9, col=1)
fig2.add_trace(go.Scatter(y=hopomo.neednectar_adult, name="neednectar_adult", mode='lines', xaxis="x", yaxis="y1"),
    row=9, col=1)
fig2.add_trace(go.Scatter(y=hopomo.neednectar_larvae, name="neednectar_larvae", mode='lines', xaxis="x", yaxis="y1"),
    row=9, col=1)

# Usages 2020
fig2.add_trace(go.Scatter(y=hopomo.usagehoney, name="usagehoney", mode='lines', xaxis="x", yaxis="y1"),
    row=10, col=1)
fig2.add_trace(go.Scatter(y=hopomo.usagenectar, name="usagenectar", mode='lines', xaxis="x", yaxis="y1"),
    row=10, col=1)
fig2.add_trace(go.Scatter(y=hopomo.usagepollen, name="usagepollen", mode='lines', xaxis="x", yaxis="y1"),
    row=10, col=1)

fig2.update_yaxes(title_text="cells", range=[0, 110000], row=1, col=1)
fig2.update_yaxes(title_text="incomes", row=2, col=1)
fig2.update_yaxes(title_text="indexes weather", nticks=10, row=3, col=1)
fig2.update_yaxes(title_text="indexes", nticks=10, row=4, col=1)
fig2.update_yaxes(title_text="foragers", nticks=10, row=5, col=1)
fig2.update_yaxes(title_text="need bees", nticks=10, row=6, col=1)
fig2.update_yaxes(title_text="larvae 1-5", nticks=10, row=7, col=1)
fig2.update_yaxes(title_text="need pollen", nticks=10, row=8, col=1)
fig2.update_yaxes(title_text="need nectar", nticks=10, row=9, col=1)
fig2.update_yaxes(title_text="usage poll & nectar", nticks=10, row=10, col=1)

fig.update_xaxes(title_text = 'Temp (°C)', ticks="outside", nticks=12, tickcolor='crimson', showline = True)

fig2.update_traces(hovertemplate="day: %{x} <br>val: %{y:.1f}")
fig2.update_layout(autosize=False, height=3000, width=1200)
fig2.show()

### **CACHE CLEARING**

Afin de ne pas réutiliser involotairement les précédents calculs, pour les résultats de la prévisions de la partie suivante, nous nettoyons le cache des calculs.

In [768]:
# Clear du cache des fonctions afin de relancer une simulation avec d'autres données

gc.collect()
  
# Collection des objets
objects = [i for i in gc.get_objects() 
           if isinstance(i, functools._lru_cache_wrapper)]
  
# "Nettoyage" des objets
for object in objects:
    object.cache_clear()

# **PREVISIONS**

Dans cette partie nous récupérons les données prévisionnelles météorologiques de Tarbes (afin d'être cohérent avec les données précédemment fournies utilisant cette position). Le requête effectuée à l'API OpenWeatherMap nous retourne un .json de la prévision météo sur les 5 prochains jours par tranche de 3h.

Ainsi, afin d'effectuer un traitement heure par heure nous complétons les données en copiant les lignes déjà présentes, en imaginant que sur les 3h la météo n'a pas changé.

## **IMPORT DES DONNÉES PASSÉES**

Petite précision concernant le nom des variables. Dans tout le notebook elles font une référence à l'année 2020 mais lors des prévisions ces variables notamment *list_data2020* se voit bien passer des données de 2021 (elle est mise à jour dans la partie suivante)

In [769]:
#Import des données des CSV provenant d'OpenDataSoft de Tarbes (ville la plus proche selectionable de Pau avec historique météorologique gratuit)
# https://public.opendatasoft.com/explore/dataset/donnees-synop-essentielles-omm/table/?refine.date=2021&refine.nom=TARBES-OSSUN&sort=-date
# L'import n'est pas direct via l'API mais passe par un import préalable du fichier csv 
# Il faut prendre jusqu'au jour du début de prévision désiré, même si un trou de données est géré lors de la modélisation des courbes avec des valeurs par défaut.
data2020 = pd.read_csv("/content/data_meteo_TARBES.csv",sep =',') 

#Données globales
dataMeteo2020 = pd.DataFrame(data2020,columns=['Date','Température','Précipitations dans la dernière heure','Humidité','Vitesse du vent moyen 10 mn'])

#Verifications du jeu de données
dataMeteo2020.isnull().sum()

# Correction du jeu de données avec un median (la correction n'est faite ici que sur les trous de données des précipitations)
# Si besoin copiez coller les deux lignes suivantes en changeant le nom de la colonne entre [ ] avec celle concernée (trou de données visibles avec l'appel précédent)
median = dataMeteo2020['Précipitations dans la dernière heure'].median()
dataMeteo2020['Précipitations dans la dernière heure'].fillna(median,inplace=True)

#Conversion Température Kelvin en Celsus
dataMeteo2020['Température'] = dataMeteo2020['Température'].apply(lambda x: round((x - 273.15),1))
#Conversion m/s en km/h de vent
dataMeteo2020['Vitesse du vent moyen 10 mn'] = dataMeteo2020['Vitesse du vent moyen 10 mn'].apply(lambda x: round((x * 3.6),1))

#Correction des valeurs négatives (défaut propre au jeu de données)
dataMeteo2020['Précipitations dans la dernière heure'].replace({-0.1 : 0}, inplace= True)

#Conversion en listes des données météo
list_DataMeteo2020 = [list(row) for row in dataMeteo2020.values]

In [770]:
# Conversion de la date/heure en deux sous listes d'entiers
convert_date(list_DataMeteo2020)

'Traitement réalisé avec succès'

In [771]:
# Mise à jour des données géorgraphiques 
# Pour la prévision on récupère les données les plus proches de Pau que nous avons trouvé, gratuites c'est à dire à Tarbes
city = LocationInfo('Tarbes', 'France', 'Europe/London', 43.232951, 0.078082)
getSunSetRise()

## **IMPORT DES DONNÉES DE PRÉVISION**

Dans cette partie nous récupérons les données météorologiques prévisionnelles de Tarbes (pour correspondre à l'historique météorologique que l'on a selectionné pour 2021).

In [772]:
# Requête OWM
forecast_url = "http://api.openweathermap.org/data/2.5/forecast?q=Tarbes,fr&units=metric&appid=2a7f4f5c2a0ae6e3fce7b81c64a61956"
 
weather_data_forecast = requests.get(forecast_url).json()

# Affichage possible des données brut JSON provenant de la requête.
# city_name = "Tarbes"
# print("\nForecast Weather Data Of " + city_name +":\n")
# pprint(weather_data_forecast)

# Conversion des données json en csv (téléchargeable ensuite en sortie)
df = pd.json_normalize(weather_data_forecast['list'])
df.to_csv("forecast_data.csv")

data_CSV = pd.read_csv("/content/forecast_data.csv",sep =',')

#Complétion des données pour obtenir du heure par heure
dataForecast_brut = pd.DataFrame(data_CSV,columns=['dt_txt','main.temp','rain.3h','main.humidity','wind.speed'])
dataForecast_brut['dt_txt'] = pd.to_datetime(dataForecast_brut['dt_txt'])
dataForecast_brut = dataForecast_brut.set_index('dt_txt').resample('H').ffill().reset_index()
dataForecast_brut['dt_txt'] = dataForecast_brut['dt_txt'].dt.strftime('%Y-%m-%d %H:%M:%S')

#Verifications du jeu de données
dataForecast_brut.isnull().sum()

# Correction du jeu de données (nan par 0)
# Comme précédemment pour l'historique, l'appel ci-dessus permet de définir les colonnes ayant des données manquantes ou non définies
# Il suffit de copier coller la ligne ci-dessous en modifiant le nom de la colonne que l'on souhaite corriger [ ]
# Ici on remplace les nan par des 0 car les nan indiques qu'il n'y a pas eu de données de pluies (présentes uniquement si de la pluie est prévue)
dataForecast_brut['rain.3h'].fillna(0,inplace=True)

#Conversion en listes des données météo
list_forecast = [list(row) for row in dataForecast_brut.values]

In [773]:
# Conversion de la date/heure en deux sous listes d'entiers
convert_date(list_forecast)

'Traitement réalisé avec succès'

In [774]:
# Récupération de la date d'aujourd'hui puis nous y soustrayons celle du premier jour de l'année :
# On obtient le delta soit l'écart en nombre de jour pour notre historique. 
today = date.today()
delta = ((today - date(2021,1,1)).days) + 1

# On redefinit le nombre de jours avec lesquels nous lanceront le modèle
# Donc on a les données d'historique (delta) + la prévision (6 jours)
nb_days = delta + 6

# On converti la date d'aujourd'hui en liste d'int pour être en cohesion avec les traitements des dates précédentes (historiques)
today_list = (today.strftime("2021,%m,%d")).split(',')
for i in range(0, len(today_list)): 
    today_list[i] = int(today_list[i])

# Fonction de recherche permettant de prendre le données météorologiques jusqu'à aujour'hui uniquement (au cas ou on a plus de données dans l'historique)
# C'était le cas lors d'un premier test ou on utilisais les données sur l'année de 2020, nous avions besoin de couper les données au jour t.today
def research():
  for i in range(len(list_DataMeteo2020)):
    if (list_DataMeteo2020[i][0] == today_list) : 
      return i

nb_elem = research()

# On concatène les données d'historique + de prévision
list_DataMeteo2020 = list_DataMeteo2020[:nb_elem] + list_forecast

## **PARTIE LPO PRÉVISIONS**

### **TRAITEMENT LPO**

In [775]:
# On lance le modèle
hopomo_prev = HOPOMO(nb_days)

In [ ]:
# On sauvegarde les résultats des fonctions dans un csv avec la date sous forme de liste pour une meilleur compréhension du fichier
hopomo_prev.to_csv("hopomo_prev.csv")
hopomo_prev.info()

### **GRAPHES PREVISIONNELS**

Les graphes suivants sont construits de la manière suivante, en deux parties : 

*   *La première partie* affiche les résultats (courbes) des différentes variables en utilisant les données d'historique.
*   *La deuxieme* séparée par un espace avec l'historique représente les prédictions sur 5 jours. Elles sont en rouge pour les graphes principaux et dans les graphes secondaires vu le nombre important de courbes nous avons laissé une génération de couleur différentes de celles de l'historique. 


Nous **vous recommendons** de vous appuyer fortement sur la **légende** à la droite des graphes ou bien **passer votre souris sur la courbe qui vous interresse pour avoir sa référence**. Chaque courbe de prevision est terminée par l'abbréviation *_prev*

#### **1. GRAPHES PRINCIPAUX**

In [777]:
fig3 = make_subplots(rows=3, cols=2, vertical_spacing=0.05)

fig3.add_trace(
    go.Scatter(y=hopomo_prev.beesadult[:delta], name="Adult Bees", mode='lines', xaxis="x", yaxis="y1", showlegend = True),
    row=1, col=1
)

fig3.add_trace(
    go.Scatter(y=hopomo_prev.beesadult[delta:], name="Adult Bees Prev", mode='lines', x0=delta, xaxis="x", yaxis="y1", line=dict(color='red'), showlegend = True),
    row=1, col=1
)

fig3.add_trace(
    go.Scatter(y=hopomo_prev.storespollen[:delta], name="Pollen Stores", mode='lines', xaxis="x", yaxis="y2",showlegend = True),
    row=1, col=2
)

fig3.add_trace(
    go.Scatter(y=hopomo_prev.storespollen[delta:], name="Pollen Stores Prev", mode='lines',x0=delta, xaxis="x", yaxis="y2", line=dict(color='red'),showlegend = True),
    row=1, col=2
)


fig3.add_trace(
    go.Scatter(y=hopomo_prev.cellsbrood[:delta], name="Brood cells", mode='lines', xaxis="x", yaxis="y3",showlegend = True),
    row=2, col=1
)

fig3.add_trace(
    go.Scatter(y=hopomo_prev.cellsbrood[delta:], name="Brood cells Prev", mode='lines',x0=delta, xaxis="x", yaxis="y3", line=dict(color='red'), showlegend = True),
    row=2, col=1
)

fig3.add_trace(
    go.Scatter(y=hopomo_prev.storeshoney[:delta], name="Honey Stores", mode='lines', xaxis="x", yaxis="y4", showlegend = True),
    row=2, col=2
)

fig3.add_trace(
    go.Scatter(y=hopomo_prev.storeshoney[delta:], name="Honey Stores Prev", mode='lines', x0=delta, xaxis="x", yaxis="y4", line=dict(color='red'), showlegend = True),
    row=2, col=2
)


fig3.add_trace(
    go.Scatter(y=hopomo_prev.weightcolony[:delta], name="Colony Weight", mode='lines', xaxis="x", yaxis="y6",showlegend = True),
    row=3, col=2
)

fig3.add_trace(
    go.Scatter(y=hopomo_prev.weightcolony[delta:], name="Colony Weight Prev", mode='lines', xaxis="x",x0=delta, yaxis="y6", line=dict(color='red'), showlegend = True),
    row=3, col=2
)

fig3.update_yaxes(title_text="adult bees", range=[0, 60000], row=1, col=1)
fig3.update_yaxes(title_text="pollen stores", range=[0,6000], row=1, col=2)
fig3.update_yaxes(title_text="brood cells", range=[0, 35000], row=2, col=1)
fig3.update_yaxes(title_text="honey stores", range=[0, 100000], row=2, col=2)
fig3.update_yaxes(title_text="colony weight (kg)", range=[0, 100], row=3, col=2)
fig3.update_yaxes(title_text="brood % comparaison", range=[0, 100], row=3, col=1)
#fig3.update_yaxes(title_text = 'Temp (°C)', ticks="outside", range=[-4, 30], tickwidth=1, nticks=30, tickcolor='crimson', showgrid=False, showline = True,secondary_y=False, spikesnap='cursor',spikethickness=1)
fig3.update_traces(hovertemplate="day: %{x} <br>val: %{y:.1f}")
fig3.update_layout(autosize=False, height=1200, width=1200)
fig3.show()


#### **2. AUTRES GRAPHES**

In [778]:
fig4 = make_subplots(rows=10, cols=1, vertical_spacing=0.005, shared_xaxes=True)

# Cells 2021
fig4.add_trace(go.Scatter(y=hopomo_prev.storeshoney[:delta], name="storeshoney", mode='lines', xaxis="x", yaxis="y1",stackgroup='one'),
    row=1, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.storesnectar[:delta], name="storesnectar", mode='lines', xaxis="x", yaxis="y1",stackgroup='one'),
    row=1, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.storespollen[:delta], name="storespollen", mode='lines', xaxis="x", yaxis="y1",stackgroup='one'),
    row=1, col=1) 
fig4.add_trace(go.Scatter(y=hopomo_prev.cellseggs[:delta], name="cellseggs", mode='lines', xaxis="x", yaxis="y1",stackgroup='one'),
    row=1, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.cellslarvae[:delta], name="cellslarvae", mode='lines', xaxis="x", yaxis="y1",stackgroup='one'),
    row=1, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.cellspupae[:delta], name="cellspupae", mode='lines', xaxis="x", yaxis="y1",stackgroup='one'),
    row=1, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.cellsbrood[:delta], name="cellsbrood", mode='lines', xaxis="x", yaxis="y1"),
    row=1, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.cellsempty[:delta], name="cellsempty", mode='lines', xaxis="x", yaxis="y1",stackgroup='one'),
    row=1, col=1)

# # Cells prevision
fig4.add_trace(go.Scatter(y=hopomo_prev.storeshoney[delta:], name="storeshoney prev", mode='lines',x0=delta, xaxis="x", yaxis="y1",stackgroup='one'),
    row=1, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.storesnectar[delta:], name="storesnectar prev", mode='lines',x0=delta, xaxis="x", yaxis="y1",stackgroup='one'),
    row=1, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.storespollen[delta:], name="storespollen prev", mode='lines',x0=delta, xaxis="x", yaxis="y1",stackgroup='one'),
    row=1, col=1) 
fig4.add_trace(go.Scatter(y=hopomo_prev.cellseggs[delta:], name="cellseggs prev", mode='lines',x0=delta, xaxis="x", yaxis="y1",stackgroup='one'),
    row=1, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.cellslarvae[delta:], name="cellslarvae prev", mode='lines',x0=delta, xaxis="x", yaxis="y1",stackgroup='one'),
    row=1, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.cellspupae[delta:], name="cellspupae prev", mode='lines',x0=delta, xaxis="x", yaxis="y1",stackgroup='one'),
    row=1, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.cellsbrood[delta:], name="cellsbrood prev", mode='lines',x0=delta, xaxis="x", yaxis="y1"),
    row=1, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.cellsempty[delta:], name="cellsempty prev", mode='lines',x0=delta, xaxis="x", yaxis="y1",stackgroup='one'),
    row=1, col=1)

#Income 2021
fig4.add_trace(go.Scatter(y=hopomo_prev.incomenectar[:delta], name="incomenectar", mode='lines', xaxis="x", yaxis="y1",stackgroup='one'),
    row=2, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.incomepollen[:delta], name="incomepollen", mode='lines', xaxis="x", yaxis="y1",stackgroup='one'),
    row=2, col=1)

# # Income prevision
fig4.add_trace(go.Scatter(y=hopomo_prev.incomenectar[delta:], name="incomenectar prev", mode='lines',x0=delta, xaxis="x", yaxis="y1",stackgroup='two'),
    row=2, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.incomepollen[delta:], name="incomepollen prev", mode='lines',x0=delta, xaxis="x", yaxis="y1",stackgroup='two'),
    row=2, col=1)

#Indexes 2021
fig4.add_trace(go.Scatter(y=hopomo_prev.indexflight[:delta], name="indexflight", mode='lines', xaxis="x", yaxis="y1"),
    row=3, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.indexrain[:delta], name="indexrain", mode='lines', xaxis="x", yaxis="y1"),
    row=3, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.indextemperature[:delta], name="index temperature", mode='lines', xaxis="x", yaxis="y1"),
    row=3, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.indexvent[:delta], name="indexvent", mode='lines', xaxis="x", yaxis="y1"),
    row=3, col=1)
# fig4.add_trace(go.Scatter(y=hopomo.indexhumidity, name="indexhumidity", mode='lines', xaxis="x", yaxis="y1"),
#     row=3, col=1)

# #Indexes prevision
fig4.add_trace(go.Scatter(y=hopomo_prev.indexflight[delta:], name="indexflight prev", mode='lines',x0=delta, xaxis="x", yaxis="y1"),
    row=3, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.indexrain[delta:], name="indexrain prev", mode='lines',x0=delta, xaxis="x", yaxis="y1"),
    row=3, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.indextemperature[delta:], name="index temperature prev", mode='lines',x0=delta, xaxis="x", yaxis="y1"),
    row=3, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.indexvent[delta:], name="indexvent prev", mode='lines',x0=delta, xaxis="x", yaxis="y1"),
    row=3, col=1)
# fig4.add_trace(go.Scatter(y=hopomo.indexhumidity, name="indexhumidity", mode='lines', xaxis="x", yaxis="y1"),
#     row=3, col=1)

# Index divers 2021
fig4.add_trace(go.Scatter(y=hopomo_prev.indexnectaroutside[:delta], name="index nectaroutside", mode='lines', xaxis="x", yaxis="y1"),
    row=4, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.indexpollenoutside[:delta], name="index pollenoutside", mode='lines', xaxis="x", yaxis="y1"),
    row=4, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.indexpollensituation[:delta], name="index pollensituation", mode='lines', xaxis="x", yaxis="y1"),
    row=4, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.indexnursingquality[:delta], name="index nursingquality", mode='lines', xaxis="x", yaxis="y1"),
    row=4, col=1)

# Index divers prévisions
fig4.add_trace(go.Scatter(y=hopomo_prev.indexnectaroutside[delta:], name="index nectaroutside prev", mode='lines',x0=delta, xaxis="x", yaxis="y1"),
    row=4, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.indexpollenoutside[delta:], name="index pollenoutside prev", mode='lines',x0=delta, xaxis="x", yaxis="y1"),
    row=4, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.indexpollensituation[delta:], name="index pollensituation prev", mode='lines',x0=delta, xaxis="x", yaxis="y1"),
    row=4, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.indexnursingquality[delta:], name="index nursingquality prev", mode='lines',x0=delta, xaxis="x", yaxis="y1"),
    row=4, col=1)

# Foragers 2021
fig4.add_trace(go.Scatter(y=hopomo_prev.foragersactive[:delta], name="foragersactive", mode='lines', xaxis="x", yaxis="y1",fill='tozeroy'),
    row=5, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.foragersnectar[:delta], name="foragersnectar", mode='lines', xaxis="x", yaxis="y1"),
    row=5, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.foragersnectaractive[:delta], name="foragersnectar active", mode='lines', xaxis="x", yaxis="y1",fill='tozeroy'),
    row=5, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.foragerspollen[:delta], name="foragerspollen", mode='lines', xaxis="x", yaxis="y1"),
    row=5, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.foragerspollenactive[:delta], name="foragerspollen active", mode='lines', xaxis="x", yaxis="y1",fill='tozeroy'),
    row=5, col=1)

# Foragers prévision
fig4.add_trace(go.Scatter(y=hopomo_prev.foragersactive[delta:], name="foragersactive prev", mode='lines',x0=delta, xaxis="x", yaxis="y1",fill='tozeroy'),
    row=5, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.foragersnectar[delta:], name="foragersnectar prev", mode='lines',x0=delta, xaxis="x", yaxis="y1"),
    row=5, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.foragersnectaractive[delta:], name="foragersnectar active prev",x0=delta, mode='lines', xaxis="x", yaxis="y1",fill='tozeroy'),
    row=5, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.foragerspollen[delta:], name="foragerspollen prev", mode='lines',x0=delta, xaxis="x", yaxis="y1"),
    row=5, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.foragerspollenactive[delta:], name="foragerspollen active prev",x0=delta, mode='lines', xaxis="x", yaxis="y1",fill='tozeroy'),
    row=5, col=1)

# Nurse and needs 2021
fig4.add_trace(go.Scatter(y=hopomo_prev.nurses[:delta], name="nurses", mode='lines', xaxis="x", yaxis="y1"),
    row=6, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.neednurses[:delta], name="neednurses", mode='lines', xaxis="x", yaxis="y1"),
    row=6, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.needworkers[:delta], name="needworkers", mode='lines', xaxis="x", yaxis="y1"),
    row=6, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.needprocessors[:delta], name="needprocessors", mode='lines', xaxis="x", yaxis="y1"),
    row=6, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.needpollenforagers[:delta], name="needpollenforagers", mode='lines', xaxis="x", yaxis="y1"),
    row=6, col=1)

# Nurse and needs prévision
fig4.add_trace(go.Scatter(y=hopomo_prev.nurses[delta:], name="nurses prev", mode='lines',x0=delta, xaxis="x", yaxis="y1"),
    row=6, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.neednurses[delta:], name="neednurses prev", mode='lines',x0=delta, xaxis="x", yaxis="y1"),
    row=6, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.needworkers[delta:], name="needworkers prev", mode='lines',x0=delta, xaxis="x", yaxis="y1"),
    row=6, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.needprocessors[delta:], name="needprocessors prev", mode='lines',x0=delta, xaxis="x", yaxis="y1"),
    row=6, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.needpollenforagers[delta:], name="needpollenforagers prev", mode='lines',x0=delta, xaxis="x", yaxis="y1"),
    row=6, col=1)

# Larvae 2021
fig4.add_trace(go.Scatter(y=hopomo_prev.larvae1[:delta], name="larvae 1", mode='lines', xaxis="x", yaxis="y1"),
    row=7, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.larvae2[:delta], name="larvae 2", mode='lines', xaxis="x", yaxis="y1"),
    row=7, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.larvae3[:delta], name="larvae 3", mode='lines', xaxis="x", yaxis="y1"),
    row=7, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.larvae4[:delta], name="larvae 4", mode='lines', xaxis="x", yaxis="y1"),
    row=7, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.larvae5[:delta], name="larvae 5", mode='lines', xaxis="x", yaxis="y1"),
    row=7, col=1)

# Larvae prévision
fig4.add_trace(go.Scatter(y=hopomo_prev.larvae1[delta:], name="larvae 1 prev", mode='lines',x0=delta, xaxis="x", yaxis="y1"),
    row=7, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.larvae2[delta:], name="larvae 2 prev", mode='lines',x0=delta, xaxis="x", yaxis="y1"),
    row=7, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.larvae3[delta:], name="larvae 3 prev", mode='lines',x0=delta, xaxis="x", yaxis="y1"),
    row=7, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.larvae4[delta:], name="larvae 4 prev", mode='lines',x0=delta, xaxis="x", yaxis="y1"),
    row=7, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.larvae5[delta:], name="larvae 5 prev", mode='lines',x0=delta, xaxis="x", yaxis="y1"),
    row=7, col=1)

# Needs pollen 2021
fig4.add_trace(go.Scatter(y=hopomo_prev.needpollen[:delta], name="needpollen", mode='lines', xaxis="x", yaxis="y1"),
    row=8, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.needpollenincome[:delta], name="needpollen income", mode='lines', xaxis="x", yaxis="y1"),
    row=8, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.needpollen_adult[:delta], name="needpollen_adult", mode='lines', xaxis="x", yaxis="y1"),
    row=8, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.needpollen_larvae[:delta], name="needpollen_larvae", mode='lines', xaxis="x", yaxis="y1"),
    row=8, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.storespollen[:delta], name="storespollen", mode='lines', xaxis="x", yaxis="y1"),
    row=8, col=1)

# Needs pollen prévision
fig4.add_trace(go.Scatter(y=hopomo_prev.needpollen[delta:], name="needpollen prev", mode='lines',x0=delta, xaxis="x", yaxis="y1"),
    row=8, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.needpollenincome[delta:], name="needpollen income prev",x0=delta, mode='lines', xaxis="x", yaxis="y1"),
    row=8, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.needpollen_adult[delta:], name="needpollen_adult prev",x0=delta, mode='lines', xaxis="x", yaxis="y1"),
    row=8, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.needpollen_larvae[delta:], name="needpollen_larvae prev",x0=delta, mode='lines', xaxis="x", yaxis="y1"),
    row=8, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.storespollen[delta:], name="storespollen prev", mode='lines',x0=delta, xaxis="x", yaxis="y1"),
    row=8, col=1)

# Needs nectar 2021
fig4.add_trace(go.Scatter(y=hopomo_prev.neednectar[:delta], name="neednectar", mode='lines', xaxis="x", yaxis="y1"),
    row=9, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.neednectar_adult[:delta], name="neednectar_adult", mode='lines', xaxis="x", yaxis="y1"),
    row=9, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.neednectar_larvae[:delta], name="neednectar_larvae", mode='lines', xaxis="x", yaxis="y1"),
    row=9, col=1)

# Needs nectar prévision
fig4.add_trace(go.Scatter(y=hopomo_prev.neednectar[delta:], name="neednectar prev", mode='lines',x0=delta, xaxis="x", yaxis="y1"),
    row=9, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.neednectar_adult[delta:], name="neednectar_adult prev", mode='lines',x0=delta, xaxis="x", yaxis="y1"),
    row=9, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.neednectar_larvae[delta:], name="neednectar_larvae prev", mode='lines',x0=delta, xaxis="x", yaxis="y1"),
    row=9, col=1)

# Usages 2021
fig4.add_trace(go.Scatter(y=hopomo_prev.usagehoney[:delta], name="usagehoney", mode='lines', xaxis="x", yaxis="y1"),
    row=10, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.usagenectar[:delta], name="usagenectar", mode='lines', xaxis="x", yaxis="y1"),
    row=10, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.usagepollen[:delta], name="usagepollen", mode='lines', xaxis="x", yaxis="y1"),
    row=10, col=1)

# Usages prévision
fig4.add_trace(go.Scatter(y=hopomo_prev.usagehoney[delta:], name="usagehoney prev", mode='lines',x0=delta, xaxis="x", yaxis="y1"),
    row=10, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.usagenectar[delta:], name="usagenectar prev", mode='lines',x0=delta, xaxis="x", yaxis="y1"),
    row=10, col=1)
fig4.add_trace(go.Scatter(y=hopomo_prev.usagepollen[delta:], name="usagepollen prev", mode='lines',x0=delta, xaxis="x", yaxis="y1"),
    row=10, col=1)


fig4.update_yaxes(title_text="cells", range=[0, 110000], row=1, col=1)
fig4.update_yaxes(title_text="incomes", row=2, col=1)
fig4.update_yaxes(title_text="indexes weather", nticks=10, row=3, col=1)
fig4.update_yaxes(title_text="indexes", nticks=10, row=4, col=1)
fig4.update_yaxes(title_text="foragers", nticks=10, row=5, col=1)
fig4.update_yaxes(title_text="need bees", nticks=10, row=6, col=1)
fig4.update_yaxes(title_text="larvae 1-5", nticks=10, row=7, col=1)
fig4.update_yaxes(title_text="need pollen", nticks=10, row=8, col=1)
fig4.update_yaxes(title_text="need nectar", nticks=10, row=9, col=1)
fig4.update_yaxes(title_text="usage poll & nectar", nticks=10, row=10, col=1)

# fig4.update_xaxes(title_text = 'Temp (°C)', ticks="outside", nticks=12, tickcolor='crimson', showline = True)

fig4.update_traces(hovertemplate="day: %{x} <br>val: %{y:.1f}")
fig4.update_layout(autosize=False, height=3000, width=1200)
fig4.show()